### 背景
- native crash具有上下文不全、出错信息模糊、难以捕捉等特点，比java crash更难修复
- 合格的异常捕获组件也要能达到以下目的：
  - 支持在 crash 时进行更多扩展操作，如
    - 打印 logcat 和应用日志
    - 上报 crash 次数
    - 对不同的 crash 做不同的恢复措施
  - 可以针对业务不断改进和适应
  
---

### 现有的方案
- 其实3个方案在 Android 平台的实现原理都是基本一致的，综合考虑，可以基于coffeecatch改进。
![image](bugly_crash_page1.png)

---

### 信号机制
#### 程序奔溃
- 在 Unix-like 系统中，所有的崩溃都是编程错误或者硬件错误相关的，系统遇到不可恢复的错误时会触发崩溃机制让程序退出，如除零、段地址错误等。
- 异常发生时，CPU 通过异常中断的方式，触发异常处理流程。不同的处理器，有不同的异常中断类型和中断处理方式。
- linux 把这些中断处理，统一为信号量，可以注册信号量向量进行处理。
- 信号机制是进程之间相互传递消息的一种方法，信号全称为软中断信号。

#### 信号机制
- 函数运行在用户态，当遇到系统调用、中断或是异常的情况时，程序会进入内核态。信号涉及到了这两种状态之间的转换。
![image](bugly_crash_page2.png)

##### 信号的接收
- 接收信号的任务是由内核代理的，当内核接收到信号后，会将其放到对应进程的信号队列中，同时向进程发送一个中断，使其陷入内核态。注意，此时信号还只是在队列中，对进程来说暂时是不知道有信号到来的。

##### 信号的检测
- 进程陷入内核态后，有两种场景会对信号进行检测：
  - 进程从内核态返回到用户态前进行信号检测
  - 进程在内核态中，从睡眠状态被唤醒的时候进行信号检测
- 当发现有新信号时，便会进入下一步，信号的处理

##### 信号的处理
- 信号处理函数是运行在用户态的，调用处理函数前，内核会将当前内核栈的内容备份拷贝到用户栈上，并且修改指令寄存器（eip）将其指向信号处理函数。
- 接下来进程返回到用户态中，执行相应的信号处理函数。
- 信号处理函数执行完成后，还需要返回内核态，检查是否还有其它信号未处理。如果所有信号都处理完成，就会将内核栈恢复（从用户栈的备份拷贝回来），同时恢复指令寄存器（eip）将其指向中断前的运行位置，最后回到用户态继续执行进程。

##### 常见信号量类型
![image](bugly_crash_page3.png)

---

### 捕捉 native crash
#### 注册信号处理函数
- 第一步就是要用信号处理函数捕获到 native crash(SIGSEGV, SIGBUS等)。在 posix 系统，可以用 sigaction()：
  - signum：代表信号编码，可以是除 SIGKILL 及 SIGSTOP 外的任何一个特定有效的信号，如果为这两个信号定义自己的处理函数，将导致信号安装错误。
  - act：指向结构体 sigaction 的一个实例的指针，该实例指定了对特定信号的处理，如果设置为空，进程会执行默认处理。
  - oldact：和参数 act 类似，只不过保存的是原来对相应信号的处理，也可设置为 NULL。
```
#include <signal.h> 
int sigaction(int signum, const struct sigaction *act, struct sigaction *oldact));
```

#### 设置额外栈空间
- SIGSEGV 很有可能是栈溢出引起的，如果在默认的栈上运行很有可能会破坏程序运行的现场，无法获取到正确的上下文。而且当栈满了（太多次递归，栈上太多对象），系统会在同一个已经满了的栈上调用SIGSEGV的信号处理函数，又再一次引起同样的信号。
- 应该开辟一块新的空间作为运行信号处理函数的栈。可以使用 sigaltstack 在任意线程注册一个可选的栈，保留一下在紧急情况下使用的空间。（系统会在危险情况下把栈指针指向这个地方，使得可以在一个新的栈上运行信号处理函数）

```
#include <signal.h>
int sigaltstack(const stack_t *ss, stack_t *oss);


stack_t stack;  
memset(&stack, 0, sizeof(stack));  
/* Reserver the system default stack size. We don't need that much by the way. */  
stack.ss_size = SIGSTKSZ;  
stack.ss_sp = malloc(stack.ss_size);  
stack.ss_flags = 0;  
/* Install alternate stack size. Be sure the memory region is valid until you revert it. */  
if (stack.ss_sp != NULL && sigaltstack(&stack, NULL) == 0) {  
  ...  
}
```

#### 兼容其他 signal 处理
- 某些信号可能在之前已经被安装过信号处理函数，而 sigaction 一个信号量只能注册一个处理函数，这意味着我们的处理函数会覆盖其他人的处理信号
- 保存旧的处理函数，在处理完我们的信号处理函数后，在重新运行老的处理函数就能完成兼容。

```
static void my_handler(const int code, siginfo_t *const si, void *const sc) {
...  
  /* Call previous handler. */  
  old_handler.sa_sigaction(code, si, sc);  
}
```

---

### 注意事项
#### 防止死锁或者死循环
- Single UNIX Specification 说明了在信号处理程序中保证调用安全的函数。这些函数是可重入的并被称为是异步信号安全（async-signal-safe）。除了可重入以外，在信号处理操作期间，它会阻塞任何会引起不一致的信号发送。
- 即使我们自己在信号处理程序中不使用不可重入的函数，也无法保证保存的旧的信号处理程序中不会有非异步信号安全的函数。所以要使用 alarm 保证信号处理程序不会陷入死锁或者死循环的状态

```
static void signal_handler(const int code, siginfo_t *const si, void *const sc) {
    /* Ensure we do not deadlock. Default of ALRM is to die.
    * (signal() and alarm() are signal-safe) */
    signal(code, SIG_DFL);
    signal(SIGALRM, SIG_DFL);
    
    /* Ensure we do not deadlock. Default of ALRM is to die.
      * (signal() and alarm() are signal-safe) */
    (void) alarm(8);
    ....
}
```

#### 在哪里打印堆栈
##### 子进程
- 一般会 clone 一个新的进程，在其中完成解析堆栈等任务。
- 下面是 Google Breakpad 的流程图，在新的进程中 DoDump，使用 ptrace 解析 crash 进程的堆栈，同时信号处理程序等待子进程完成任务后，再调用旧的信号处理函数。父子进程使用管道通信。
![image](bugly_crash_page4.png)

##### 子线程
- 在初始化的时候就建立了子线程并一直等待，等到捕捉到 crash 信号时，唤醒这条线程 dump 出 crash 堆栈，并把 crash 堆栈回调给 java

```
static void nativeInit(JNIEnv* env, jclass javaClass, jstring packageNameStr, jstring tombstoneFilePathStr, jobject obj) {
    ...
    initCondition();

    pthread_t thd;
    int ret = pthread_create(&thd, NULL, DumpThreadEntry, NULL);    if(ret) {
        qmlog("%s", "pthread_create error");
    }
}

void* DumpThreadEntry(void *argv) {
    JNIEnv* env = NULL;
     if((*g_jvm)->AttachCurrentThread(g_jvm, &env, NULL) != JNI_OK)
    {
        LOGE("AttachCurrentThread() failed");
        estatus = 0;
        return &estatus;
    }
        
    while (true) {
        //等待信号处理函数唤醒
        waitForSignal();

        //回调native异常堆栈给java层
        throw_exception(env);
        
        //告诉信号处理函数已经处理完了
        notifyThrowException();
    }
    
    if((*g_jvm)->DetachCurrentThread(g_jvm) != JNI_OK)
    {
        LOGE("DetachCurrentThread() failed");
        estatus = 0;
        return &estatus;
    }

    return &estatus;
}
```

---

### 收集 native crash 原因
```
/*信号处理函数*/

void (*sa_sigaction)(const int code, siginfo_t *const si, void * const sc) 

siginfo_t {
   int      si_signo;     /* Signal number 信号量 */
   int      si_errno;     /* An errno value */
   int      si_code;      /* Signal code 错误码 */
   }
```

#### code
- 发生 native crash 之后，logcat 中会打出如下一句信息

```
signal 11 (SIGSEGV), code 0 (SI_USER), fault addr 0x0
```

![image](bugly_crash_page5.png)

- 部分代码
```
  case SIGSEGV:
    switch(code) {
    case SEGV_MAPERR:
      return "Address not mapped to object";
    case SEGV_ACCERR:
      return "Invalid permissions for mapped object";
    default:
      return "Segmentation violation";
    }
    break;
```

#### pc 值
- 信号处理函数中的第三个入参 sc 是 uc_mcontext 的结构体，是 cpu 相关的上下文，包括当前线程的寄存器信息和奔溃时的 pc 值。能够知道崩溃时的 pc，就能知道崩溃时执行的是那条指令。
  - x86-64架构：uc_mcontext.gregs[REG_RIP]
  - arm架构：uc_mcontext.arm_pc

#### 共享库名字和相对偏移地址
- dladdr()
  - pc 值是程序加载到内存中的绝对地址，我们需要拿到奔溃代码相对于共享库的相对偏移地址，才能使用 addr2line 分析出是哪一行代码。通过dladdr() 可以获得共享库加载到内存的起始地址，和 pc 值相减就可以获得相对偏移地址，并且可以获得共享库的名字

```
Dl_info info;  
if (dladdr(addr, &info) != 0 && info.dli_fname != NULL) {  
  void * const nearest = info.dli_saddr;  
  //相对偏移地址
  const uintptr_t addr_relative =  
    ((uintptr_t) addr - (uintptr_t) info.dli_fbase);  
  ...  
}
```

- Linux 下进程的地址空间布局
  - 上图中 Random stack offset 和 Random mmap offset等随机值意在防止恶意程序
  - Linux 通过对栈、内存映射段、堆的起始地址加上随机偏移量来打乱布局，以免恶意程序通过计算访问栈、库函数等地址
  - 栈(stack)，作为进程的临时数据区，增长方向是从高地址到低地址
![image](bugly_crash_page6.png)

- /proc/self/maps：检查各个模块加载在内存的地址范围
  - 保存了各个程序段在内存中的加载地址范围
  - grep 出共享库的名字，就可以知道共享库的加载基值是多少
  - 得到相对偏移地址之后，使用 readelf 查看共享库的符号表，就可以知道是哪个函数 crash 了

![image](bugly_crash_page7.png)
![image](bugly_crash_page8.png)

---

### 获取堆栈
#### 原理
- 在前一步，我们获取了奔溃时的 pc 值和各个寄存器的内容，通过 SP 和 FP 所限定的 stack frame，就可以得到母函数的 SP 和 FP，从而得到母函数的 stack frame（PC，LR，SP，FP 会在函数调用的第一时间压栈），以此追溯，即可得到所有函数的调用顺序
![image](bugly_crash_page9.png)

#### 实现
- 在 4.1.1 以上，5.0 以下：使用安卓系统自带的libcorkscrew.so
- 在 5.0 以上：安卓系统中没有了 libcorkscrew.so，使用自己编译的 libunwind
  - libunwind 是一个独立的开源库，高版本的安卓源码中也使用了 libunwind 作为解堆栈的工具，并针对安卓做了一些适配

```
#ifdef USE_UNWIND
    /* Frame buffer initial position. */
    t->frames_size = 0;
    
    /* Skip us and the caller. */
    t->frames_skip = 0;
    
    /* 使用libcorkscrew解堆栈 */
#ifdef USE_CORKSCREW
    t->frames_size = backtrace_signal(si, sc, t->frames, 0, BACKTRACE_FRAMES_MAX);
#else
    /* Unwind frames (equivalent to backtrace()) */
    _Unwind_Backtrace(coffeecatch_unwind_callback, t);
#endif
    
/* 如果无法加载libcorkscrew，则使用自己编译的libunwind解堆栈 */

#ifdef USE_LIBUNWIND
    if (t->frames_size == 0) {
        size_t i;
        t->frames_size = unwind_signal(si, sc, t->uframes, 0,BACKTRACE_FRAMES_MAX);
        for(i = 0 ; i < t->frames_size ; i++) {
            t->frames[i].absolute_pc = (uintptr_t) t->uframes[i];
            t->frames[i].stack_top = 0;
            t->frames[i].stack_size = 0;
            __android_log_print(ANDROID_LOG_DEBUG, TAG, "absolute_pc:%x", t->frames[i].absolute_pc);
        }
    }
#endif
```

- 使用 libunwind 解堆栈的主循环，每次循环解一层堆栈

```
static ALWAYS_INLINE int
slow_backtrace (void **buffer, int size, unw_context_t *uc)
{
  unw_cursor_t cursor;
  unw_word_t ip;
  int n = 0;
  
  if (unlikely (unw_init_local (&cursor, uc) < 0))
    return 0;
    
  while (unw_step (&cursor) > 0)
    {
      if (n >= size)
          return n;
      if (unw_get_reg (&cursor, UNW_REG_IP, &ip) < 0)
          return n;
      buffer[n++] = (void *) (uintptr_t) ip;
    }
  return n;
}
```

#### 获取函数符号
- libcorkscrew
  - 通过 libcorkscrew 中的 get_backtrace_symbols 函数获得函数符号

```
/*
* Describes the symbols associated with a backtrace frame.
*/
typedef struct {
    uintptr_t relative_pc;
    uintptr_t relative_symbol_addr;
    char* map_name;
    char* symbol_name;    
    char* demangled_name;
} backtrace_symbol_t;
    
/*
* Gets the symbols for each frame of a backtrace.
* The symbols array must be big enough to hold one symbol record per frame.
* The symbols must later be freed using free_backtrace_symbols.
*/

void get_backtrace_symbols(const backtrace_frame_t* backtrace, size_t frames,
        backtrace_symbol_t* backtrace_symbols);
```

- dladdr
  - 更通用的方法是通过 dladdr 获得函数名字

```
int dladdr(void *addr, Dl_info *info);

typedef struct {
   const char *dli_fname;  /* Pathname of shared object that
                              contains address */
   void       *dli_fbase;  /* Base address at which shared
                              object is loaded */
   const char *dli_sname;  /* Name of symbol whose definition
                              overlaps addr */
   void       *dli_saddr;  /* Exact address of symbol named
                              in dli_sname */
} Dl_info;
```

---

### 获得 java 堆栈
- 在信号处理函数中获得当前线程的名字，然后把 crash 线程的名字传给 java 层，在 java 里 dump 出这个线程的堆栈，就是 crash 所对应的 java 层堆栈了

```
char* getThreadName(pid_t tid) {
    if (tid <= 1) {
        return NULL;
    }
    char* path = (char *) calloc(1, 80);
    char* line = (char *) calloc(1, THREAD_NAME_LENGTH);

    snprintf(path, PATH_MAX, "proc/%d/comm", tid);
    FILE* commFile = NULL;
    if (commFile = fopen(path, "r")) {
        fgets(line, THREAD_NAME_LENGTH, commFile);
        fclose(commFile);
    }
    free(path);
    if (line) {
        int length = strlen(line);
        if (line[length - 1] == '\n') {
            line[length - 1] = '\0';
        }
    }
    return line;
}
```

```
/**
 * 根据线程名获得线程对象，native 层会调用该方法，不能混淆
 * @param threadName
 * @return
 */
@Keep
public static Thread getThreadByName(String threadName) {
    if (TextUtils.isEmpty(threadName)) {
        return null;
    }

    Set<Thread> threadSet = Thread.getAllStackTraces().keySet();
    Thread[] threadArray = threadSet.toArray(new Thread[threadSet.size()]);

    Thread theThread = null;
    for(Thread thread : threadArray) {
        if (thread.getName().equals(threadName)) {
            theThread =  thread;
        }
    }

    Log.d(TAG, "threadName: " + threadName + ", thread: " + theThread);
    return theThread;
}
```

---

### 参考
[参考文档](https://mp.weixin.qq.com/s/g-WzYF3wWAljok1XjPoo7w?)